# Income Fluctuation Problem (Bewley/Aiyagari)

The **income fluctuation problem** is the building block of heterogeneous agent models
in modern macroeconomics. An infinitely-lived consumer facing stochastic income chooses
how much to consume and save subject to a borrowing constraint. The consumer's problem
is governed by the Bellman equation:

$$V(a, y) = \max_{c} \left\{ u(c) + \beta \, \mathbb{E}\left[ V(a', y') \mid y \right] \right\}$$

where the budget constraint and borrowing limit are:

$$a' = (1 + r)a + y - c, \qquad a' \geq -b$$

Here $a$ is the asset level, $y$ is stochastic income (typically an AR(1) in logs),
$r$ is the interest rate, $b$ is the borrowing limit, and $\beta$ is the discount factor.
The utility function $u(c) = \frac{c^{1-\gamma}}{1-\gamma}$ exhibits constant relative risk aversion.

This model captures the precautionary savings motive: risk-averse agents accumulate
a buffer stock of assets to self-insure against income shocks. It forms the core of
Bewley (1986), Huggett (1993), and Aiyagari (1994) models that are central to the
study of wealth inequality and macroeconomic policy.

In [ ]:
import bellmaneq
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## Solving the Model

In [ ]:
result = bellmaneq.solve_income_fluctuation(discount=0.95, risk_aversion=2.0, n_a=50, n_y=7)

print(f"Converged: {result.converged}")
print(f"Iterations: {result.iterations}")
print(f"Value function shape: {result.get_values().shape}")

## Visualization

In [ ]:
from bellmaneq.viz.econ import plot_income_fluctuation

fig = plot_income_fluctuation(
    result.get_asset_grid(),
    result.get_income_grid(),
    result.get_values(),
    result.get_policy(),
    result.get_savings_policy(),
)
plt.show()

## Borrowing Constraint Effects

The borrowing constraint $a' \geq -b$ plays a crucial role in the income fluctuation model.
When borrowing is prohibited ($b = 0$), agents cannot smooth consumption by borrowing
against future income, which amplifies the precautionary savings motive. Relaxing the
constraint ($b = 1$) allows agents to borrow, reducing buffer-stock savings but
introducing the risk of hitting the borrowing limit during bad income realizations.

We compare the consumption and savings policies under these two regimes.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for b, label in [(0.0, "b=0 (no borrowing)"), (1.0, "b=1")]:
    r = bellmaneq.solve_income_fluctuation(discount=0.95, borrowing_limit=b, n_a=50, n_y=7)
    a_grid = r.get_asset_grid()
    y_grid = r.get_income_grid()
    mid_y = len(y_grid) // 2
    axes[0].plot(a_grid, r.get_policy()[:, mid_y], label=label)
    axes[1].plot(a_grid, r.get_savings_policy()[:, mid_y], label=label)

axes[0].set_xlabel("Assets (a)")
axes[0].set_ylabel("Consumption")
axes[0].set_title("Consumption Policy (median income)")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

lo, hi = a_grid.min(), a_grid.max()
axes[1].plot([lo, hi], [lo, hi], 'k--', alpha=0.5, label="45-degree")
axes[1].set_xlabel("Current Assets (a)")
axes[1].set_ylabel("Next-Period Assets (a')")
axes[1].set_title("Savings Policy (median income)")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

fig.suptitle("Effect of Borrowing Constraint", fontsize=14)
fig.tight_layout()
plt.show()

## Interest Rate Sensitivity

The interest rate $r$ governs the return on savings and affects the trade-off between
current and future consumption. A higher interest rate makes saving more attractive
(substitution effect) but also increases lifetime wealth (income effect). We compare
policies across three interest rate levels: $r = 0.01$, $r = 0.03$, and $r = 0.05$.

In [ ]:
rates = [0.01, 0.03, 0.05]
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for rate in rates:
    r = bellmaneq.solve_income_fluctuation(
        discount=0.95, risk_aversion=2.0, interest_rate=rate, n_a=50, n_y=7
    )
    a_grid = r.get_asset_grid()
    y_grid = r.get_income_grid()
    mid_y = len(y_grid) // 2
    axes[0].plot(a_grid, r.get_policy()[:, mid_y], label=f"r={rate}")
    axes[1].plot(a_grid, r.get_savings_policy()[:, mid_y], label=f"r={rate}")
    print(f"r={rate}: converged={r.converged}, iterations={r.iterations}")

axes[0].set_xlabel("Assets (a)")
axes[0].set_ylabel("Consumption")
axes[0].set_title("Consumption Policy (median income)")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

lo, hi = a_grid.min(), a_grid.max()
axes[1].plot([lo, hi], [lo, hi], 'k--', alpha=0.5, label="45-degree")
axes[1].set_xlabel("Current Assets (a)")
axes[1].set_ylabel("Next-Period Assets (a')")
axes[1].set_title("Savings Policy (median income)")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

fig.suptitle("Effect of Interest Rate", fontsize=14)
fig.tight_layout()
plt.show()

## Convergence

In [ ]:
from bellmaneq.viz import plot_convergence

history = result.get_convergence_history()

fig = plot_convergence(
    history,
    title="Income Fluctuation Problem Convergence",
)
plt.show()